In [2]:
# Bari - Desafio de Estágio
# Mateus Luiz Bailo - Email: mateus_bailo@hotmail.com

# -------------------------------
# 1. Carregamento de Bibliotecas
# -------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sqlalchemy import create_engine
from dotenv import load_dotenv


In [3]:
# --------------------------
# 2. Conexão com o Banco de Dados MySQL
# --------------------------

load_dotenv()

usuario = os.getenv("DB_USER")
senha = os.getenv("DB_PASSWORD").replace("@", "%40")
host = os.getenv("DB_HOST")
porta = os.getenv("DB_PORT")
banco = os.getenv("DB_NAME")



conexao = create_engine(f"mysql+mysqlconnector://{usuario}:{senha}@{host}:{porta}/{banco}")

In [5]:
# --------------------------
# 3. Consulta SQL: Carregamento dos Dados
# --------------------------

query = """
    SELECT id_cliente as IdCliente
          ,idade AS Idade
          ,renda_mensal as RendaMensal
          ,score_credito as ScoreCredito
          ,valor_imovel as ValorImovel
          ,prazo_contrato_meses as ContratoMeses
          ,valor_financiado as ValorFinanciado
          ,inadimplente as Inadimplente
    FROM clientes_credito 
"""

df_sql = pd.read_sql(query, conexao)
print(df_sql.head())

   IdCliente  Idade  RendaMensal  ScoreCredito  ValorImovel  ContratoMeses  \
0          1     59     13921.40           548    463771.95            360   
1          2     49      6939.90           825    444990.02            360   
2          3     35      7116.43           761    489861.78            300   
3          4     63      2933.34           312    263163.69            300   
4          5     28      9427.98           683    419948.53            120   

   ValorFinanciado  Inadimplente  
0        255471.75             0  
1        194613.60             0  
2        305831.89             0  
3        173892.50             1  
4        307163.49             0  


In [14]:
# --------------------------
# 4. Análise Exploratória
# --------------------------
print("-------------Informações sobre as variaveis-------------")
print(df_sql.info())
print("-------------Informações sobre a tabela-------------")
print(df_sql.describe())
print("-------------Quantidade de Null's-------------")
print(df_sql.isnull().sum())

-------------Informações sobre as variaveis-------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   IdCliente        2000 non-null   int64  
 1   Idade            2000 non-null   int64  
 2   RendaMensal      2000 non-null   float64
 3   ScoreCredito     2000 non-null   int64  
 4   ValorImovel      2000 non-null   float64
 5   ContratoMeses    2000 non-null   int64  
 6   ValorFinanciado  2000 non-null   float64
 7   Inadimplente     2000 non-null   int64  
dtypes: float64(3), int64(5)
memory usage: 125.1 KB
None
-------------Informações sobre a tabela-------------
         IdCliente        Idade   RendaMensal  ScoreCredito    ValorImovel  \
count  2000.000000  2000.000000   2000.000000   2000.000000    2000.000000   
mean   1000.500000    45.235000   8106.486620    575.658500  395943.821845   
std     577.494589    14.044942  

In [ ]:
# --------------------------
# 4. Análise Exploratória
# --------------------------

query_clientes_risco = """
SELECT id_cliente as IdCliente
      ,idade as Idade
      ,renda_mensal as RendaMensal
      ,score_credito as ScoreCredito
      ,inadimplente as Inadimplente
FROM clientes_credito
WHERE score_credito < 500 AND inadimplente = 1
"""

df_risco = pd.read_sql(query_clientes_risco, conexao)
print("Clientes com alto risco:")
print(df_risco) # 145 Clientes de Risco

Clientes com alto risco:
     IdCliente  Idade  RendaMensal  ScoreCredito  Inadimplente
0            4     63      2933.34           312             1
1           23     22      8071.49           463             1
2           34     36      4592.40           493             1
3           56     28      3934.46           303             1
4           57     67     12291.91           392             1
..         ...    ...          ...           ...           ...
140       1920     31      9108.18           384             1
141       1941     30      3993.31           448             1
142       1946     53      8549.46           473             1
143       1979     28      5039.24           372             1
144       2000     33     10726.87           381             1

[145 rows x 5 columns]
